![](https://www.zorgkaartnederland.nl/bundles/zkncontent/images/elements/logo_zkn.svg)

In [1]:
import zorgkaart

In [2]:
import folium
import os
import pandas as pd
from ipywidgets import *

## Demo: Wachttijden ziekenhuizen

In [3]:
if not ('ziekenhuis_wachttijden.csv' in os.listdir('data') and 'ziekenhuis.csv' in os.listdir('data')):
    zorgkaart.get_info('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.get_details('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.merge_datasets('Ziekenhuis', delete=True)
    zorgkaart.get_wachttijden('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)

In [4]:
wachttijden = pd.read_csv(os.path.join('data', 'ziekenhuis_wachttijden.csv'), index_col='id')
ziekenhuis = pd.read_csv(os.path.join('data', 'ziekenhuis.csv'), index_col='id')
df = pd.merge(ziekenhuis, wachttijden)

In [5]:
def wachttijden(specialisme, Treeknorm):
    df_specialisme = df[df.specialisme==specialisme]
    m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)
    for row in df_specialisme.iterrows():
        row_values = row[1]
        location = (row_values.latitude, row_values.longitude)
        inst = row_values.zorginstelling
        adr1 = row_values.adres
        adr2 = row_values.postcode + '  ' + row_values.plaats
        tel = row_values.telefoon
        bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
        web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
        wtijd = row_values.wachttijd
        wtext = 'Wachttijd: {} weken'.format(wtijd)
        popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw, wtext)
        if wtijd <= Treeknorm:
            icon=folium.Icon(color='green', icon='info-sign')
        else:
            icon=folium.Icon(color='red', icon='info-sign')
        marker = folium.Marker(location = location, popup = popup, icon=icon)
        marker.add_to(m)
    return display(m)

In [6]:
interact(wachttijden,
         specialisme = widgets.Dropdown(options=sorted(df.specialisme.unique()), description='Specialisme'),
         Treeknorm = widgets.IntSlider(4, 0, 20, description='Treeknorm'))

interactive(children=(Dropdown(description='Specialisme', options=('Allergologie', 'Cardiologie', 'Dermatologi…

<function __main__.wachttijden(specialisme, Treeknorm)>

## Delete data folder

In [7]:
try:
    for file in os.listdir('data'):
        os.remove(os.path.join('data', file))
    os.rmdir('data')
except:
    pass

## Organisatietype
Met de functie `zorgkaart.get_types()` krijg je een overzicht van alle organisatietypen waarvoor je met de module `zorgkaart` data kan ophalen van de websit [www.zorkaartnederland.nl](www.zorkaartnederland.nl).  

De variable `aantal` geeft het aantal zorginstellingen aan in betreffende categorie.

In [8]:
organisatietypen = zorgkaart.get_types()
organisatietypen.head(30)

,organisatietype,aantal
organisatietype_id,,
abortuskliniek,Abortuskliniek,14
apotheek,Apotheek,1900
arbo-en-reintegratiebedrijf,Arbo- en reïntegratiebedrijf,539
artsenpraktijk-voor-alternatieve-geneeskunde,Artsenpraktijk voor alternatieve geneeskunde,420
astmacentrum,Astmacentrum,8
audiologisch-centrum,Audiologisch centrum,50
belangenorganisatie,Belangenorganisatie,29
besnijdeniscentrum,Besnijdeniscentrum,23
bevolkingsonderzoek-kanker,Bevolkingsonderzoek kanker,5


## Info

In [9]:
info = zorgkaart.get_info('Astmacentrum')
info

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


,zorginstelling,plaats,beoordeling,waarderingen,categorie,latitude,longitude,zknl_url
id,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Clavadel,9.2,107,Astmacentrum,0.000000,0.000000,https://www.zorgkaartnederland.nl/zorginstelli...
3058759,Center of Excellence,Amersfoort,9.0,25,Astmacentrum,52.165986,5.407762,https://www.zorgkaartnederland.nl/zorginstelli...
109628,"Merem, locatie Hilversum",Hilversum,8.8,15,Astmacentrum,52.212541,5.201926,https://www.zorgkaartnederland.nl/zorginstelli...
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Horn,8.2,10,Astmacentrum,51.219635,5.918302,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Organisatie,51.801576,5.894875,https://www.zorgkaartnederland.nl/zorginstelli...
3051521,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.817159,5.831195,https://www.zorgkaartnederland.nl/zorginstelli...
3051520,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.823384,5.860092,https://www.zorgkaartnederland.nl/zorginstelli...
3048242,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Astmacentrum,51.801910,5.898534,https://www.zorgkaartnederland.nl/zorginstelli...


In [10]:
os.listdir('data')

['astmacentrum_info.csv']

## Details

In [11]:
details = zorgkaart.get_details('Astmacentrum')
details

100%|██████████| 8/8 [00:18<00:00,  2.12s/it]


,zorginstelling,adres,postcode,plaats,telefoon,website,wachttijden_url
id,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",https://nadavos.nl/,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,http://www.coe-amersfoort.nl,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,http://www.merem.nl,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,http://www.raack.org,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,c,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN


In [12]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

## Samenvoegen

In [13]:
samen = zorgkaart.merge_datasets('Astmacentrum', delete=False)
samen

,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
id,,,,,,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",9.2,107,Astmacentrum,0.000000,0.000000,https://nadavos.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,9.0,25,Astmacentrum,52.165986,5.407762,http://www.coe-amersfoort.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,8.8,15,Astmacentrum,52.212541,5.201926,http://www.merem.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,8.2,10,Astmacentrum,51.219635,5.918302,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Organisatie,51.801576,5.894875,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,NaN,0,Astmacentrum,51.817159,5.831195,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,NaN,0,Astmacentrum,51.823384,5.860092,c,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Astmacentrum,51.801910,5.898534,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


In [14]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum.csv', 'astmacentrum_info.csv']

In [15]:
os.remove(os.path.join('data', 'astmacentrum.csv'))
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

In [16]:
samen = zorgkaart.merge_datasets('Astmacentrum', delete=True)
os.listdir('data')

['astmacentrum.csv']

## Meerdere organisatietypen

In [17]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [18]:
lijst = organisatietypen[organisatietypen.aantal <= 10]
lijst

,organisatietype,aantal
organisatietype_id,,
astmacentrum,Astmacentrum,8
bevolkingsonderzoek-kanker,Bevolkingsonderzoek kanker,5
bloedbank,Bloedbank,6
brandwondencentrum,Brandwondencentrum,3
ivf-kliniek,IVF-kliniek,10
ministerie,Ministerie,1
orthoptistenpraktijk,Orthoptistenpraktijk,4


In [19]:
for organisatietype in lijst.organisatietype:
    zorgkaart.get_info(organisatietype, 1)
    zorgkaart.get_details(organisatietype, 1)
    zorgkaart.merge_datasets(organisatietype, delete=True)

100%|██████████| 4/4 [00:05<00:00,  1.38s/it]


In [20]:
os.listdir('data')

['orthoptistenpraktijk.csv',
 'bevolkingsonderzoek_kanker.csv',
 'bloedbank.csv',
 'brandwondencentrum.csv',
 'ivf_kliniek.csv',
 'astmacentrum.csv',
 'ministerie.csv']

In [21]:
data = pd.read_csv(os.path.join('data', os.listdir('data')[0]))
for file in os.listdir('data')[1:]:
    data = data.append(pd.read_csv(os.path.join('data', file)))
data.reset_index(drop=True, inplace=True)

In [22]:
data.sample(n=10)

,id,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
27,3023853,"Leids Universitair Medisch Centrum (LUMC), Voo...",Albinusdreef 2,2333 ZA,Leiden,071-526 28 70,NaN,0,IVF-kliniek,52.166763,4.477523,https://www.lumc.nl/org/voortplantingscentrum/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
9,102414,"Sanquin Bloedvoorziening, regio Zuidwest, loca...",Wytemaweg 10,3015 CN,Rotterdam,010-4630630,10.0,1,Bloedbank,51.912872,4.469369,http://www.sanquin.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
23,3023852,Isala Fertiliteitscentrum,Dokter Spanjaardweg 29,8025 BT,Zwolle,038-4245224,6.9,27,IVF-kliniek,52.511279,6.127360,http://www.isala.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
10,10001935,Sanquin Bloedvoorziening,Plesmanlaan 125,1066 CX,Amsterdam,020-5123000,8.2,2,Organisatie,52.351268,4.815259,http://www.sanquin.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
6,3020667,Bevolkingsonderzoek Noord,Queridolaan 5,9721 SZ,Groningen,0900-2030100,NaN,0,Bevolkingsonderzoek kanker,53.191495,6.571183,http://www.bevolkingsonderzoeknoord.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
2,3059929,Orthoptistenpraktijk Bastiaans Optiek,Oudedijk 143,3061 AA,Rotterdam,010-4122327,NaN,0,Orthoptistenpraktijk,51.927423,4.509913,http://www.bastiaansoptiek.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
26,3023851,Voortplantingsgeneeskunde Erasmus MC,Dr. Molewaterplein 40,3015 GD,Rotterdam,010-7040116,3.1,8,IVF-kliniek,51.910168,4.466608,http://www.erasmusmc.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
29,3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,9.0,25,Astmacentrum,52.165986,5.407762,http://www.coe-amersfoort.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
8,3038479,Bevolkingsonderzoek Zuid-West,Maasstadweg 124,3079 DZ,Rotterdam,088-2482100,NaN,0,Bevolkingsonderzoek kanker,51.879950,4.535390,http://www.bevolkingsonderzoekzuid-west.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
11,102170,"Sanquin Bloedvoorziening, regio Zuidwest, loca...",Plesmanlaan 1 A,2333 BZ,Leiden,071-5685000,6.3,1,Bloedbank,52.164290,4.477660,http://www.sanquin.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


## Command line interface

In [23]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [24]:
!python zorgkaart.py "Centrum voor bijzondere tandheelkunde"

100%|███████████████████████████████████████████| 32/32 [01:06<00:00,  2.30s/it]


In [25]:
os.listdir('data')

['centrum_voor_bijzondere_tandheelkunde.csv']

## Zorgkaart

In [26]:
df = pd.read_csv(os.path.join('data', 'centrum_voor_bijzondere_tandheelkunde.csv'), index_col='id')
df.sample(n=5)

,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
id,,,,,,,,,,,,,
3022211,Centrum voor Bijzondere Tandheelkunde Friesland,Henri Dunantweg 2,8934 AD,Leeuwarden,058-2866128,NaN,0,Centrum voor bijzondere tandheelkunde,53.189431,5.805663,http://www.mcl.nl/cbt,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3022205,Centrum voor Bijzondere Tandheelkunde Breda,Molengracht 21,4818 CK,Breda,076-5953042,8.8,2,Centrum voor bijzondere tandheelkunde,51.581441,4.798579,http://www.amphia.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3022221,Centrum voor Bijzondere Tandheelkunde Vlissingen,Koudekerkseweg 88,4382 EE,Vlissingen,0118-425522,NaN,0,Centrum voor bijzondere tandheelkunde,51.451402,3.565289,http://www.kaakchirurgiezeeland.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
10001218,Centrum Bijzondere Tandheelkunde Amarant,Bredaseweg 570,5036 NB,Tilburg,013-4645479,NaN,0,Centrum voor bijzondere tandheelkunde,51.561294,5.002744,https://cbtamarant.tandartsennet.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
184408,Centrum Bijzondere Tandheelkunde Midden-Brabant,Dorpsplein 13,4851 CP,Ulvenhout,076-5933213,NaN,0,Centrum voor bijzondere tandheelkunde,51.552574,4.797456,https://www.cbtmb.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


In [27]:
# Create map
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)

# Add marker for each gym
for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    inst = row_values.zorginstelling
    adr1 = row_values.adres
    adr2 = row_values.postcode + '  ' + row_values.plaats
    tel = row_values.telefoon
    bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
    web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
    popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw)
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(m)

# Display map
display(m)

## Wachttijden ziekenhuizen

In [28]:
if not ('ziekenhuis_wachttijden.csv' in os.listdir('data') and 'ziekenhuis.csv' in os.listdir('data')):
    zorgkaart.get_info('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.get_details('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.merge_datasets('Ziekenhuis', delete=True)
    zorgkaart.get_wachttijden('Ziekenhuis', 0) # warning: unreasonable rate (reasonable rate off)

100%|██████████| 308/308 [01:59<00:00,  2.73it/s]


In [29]:
wachttijden = pd.read_csv(os.path.join('data', 'ziekenhuis_wachttijden.csv'), index_col='id')
ziekenhuis = pd.read_csv(os.path.join('data', 'ziekenhuis.csv'), index_col='id')
df = pd.merge(ziekenhuis, wachttijden)

In [30]:
def wachttijden(specialisme, Treeknorm):
    df_specialisme = df[df.specialisme==specialisme]
    m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)
    for row in df_specialisme.iterrows():
        row_values = row[1]
        location = (row_values.latitude, row_values.longitude)
        inst = row_values.zorginstelling
        adr1 = row_values.adres
        adr2 = row_values.postcode + '  ' + row_values.plaats
        tel = row_values.telefoon
        bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
        web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
        wtijd = row_values.wachttijd
        wtext = 'Wachttijd: {} weken'.format(wtijd)
        popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw, wtext)
        if wtijd <= Treeknorm:
            icon=folium.Icon(color='green', icon='info-sign')
        else:
            icon=folium.Icon(color='red', icon='info-sign')
        marker = folium.Marker(location = location, popup = popup, icon=icon)
        marker.add_to(m)
    return display(m)

In [31]:
interact(wachttijden,
         specialisme = widgets.Dropdown(options=sorted(df.specialisme.unique()), description='Specialisme'),
         Treeknorm = widgets.IntSlider(4, 0, 20, description='Treeknorm'))

interactive(children=(Dropdown(description='Specialisme', options=('Allergologie', 'Cardiologie', 'Dermatologi…

<function __main__.wachttijden(specialisme, Treeknorm)>